In [3]:
import numpy as np
import matplotlib.pyplot as plt

D'abord on va lire les données stockées dans un fichier

In [ ]:
# Load all the data
XY = np.loadtxt('Data1.txt')

# figure
plt.title("Données")
plt.plot(XY[:,0],XY[:,1],'.')
plt.xlabel('x')
plt.ylabel('y')

Base Spline d'ordre un (fonction chapeau)

In [ ]:
def hat(xl,xr,xi,x):
    if x>xr or x<xl:
        return 0.
    elif x>=xl and x<xi:
        return (x-xl)/(xi-xl)
    elif x>xi and x<=xr: 
        return (x-xr)/(xi-xr)
    else:
        return 1.0


def base_spline1(i,Xi,X):
    xl = Xi[max(0,i-1)]
    xi = Xi[i]
    xr = Xi[min(len(Xi)-1,i+1)]
    return np.array([hat(xl,xr,xi,x) for x in X])


# test
x = np.linspace(0.0, 1.0, 100)
x_nodes = np.array([0.,0.25,0.5,0.75,1.0])

for i in range(5):
    plt.plot(x, base_spline1(i, x_nodes,x), label= 's{0}'.format(i))
plt.grid()
plt.xlabel('x')
plt.ylabel('y')
plt.legend()

Interpolation par morceaux (spline linéaire)

In [ ]:
def interp_spline1(Xi,Yi,X):
    p = 0.
    for i in range(len(Xi)):
        si = base_spline1(i,Xi,X)
        p += Yi[i]*si

    return p

class InterpSpline1:
    def __init__(self, Xi, Yi):
        self.nsplit = len(Xi)-1
        self.Xi = Xi
        self.Yi = Yi

    def __call__(self, x):
        return interp_spline1(self.Xi,self.Yi,x)

# Test    
x = np.linspace(0.0,3.0, 100)
x_nodes = np.array([0.,1.0,2.0,3.0])
y_nodes = np.array([1.0,1.5,3.0,3.5])
p = InterpSpline1(x_nodes,y_nodes)
plt.plot(x,p(x))
plt.plot(x_nodes, y_nodes, 'o')
plt.grid()

In [62]:
min_nodes = 3
max_nodes = 80
poly_list = []

for k in range(min_nodes,max_nodes+1,2):
    ii = np.linspace(0,len(XY)-1,k).astype('int')            # Position of the nodes in the data vector
    XYii=XY[ii,:]    # XY values of the nodes
    poly_list.append(InterpSpline1(XYii[:,0],XYii[:,1]))

In [ ]:
x = np.linspace(np.min(XY[:,0]), np.max(XY[:,0]), 100)

for p in poly_list:
    plt.plot(x,p(x), label = 's{0}'.format(p.nsplit))

plt.plot(XY[:,0],XY[:,1], '.', label = 'données')
plt.legend(loc='best')
plt.grid()


In [ ]:
error = []
for p in poly_list:
    error.append(np.linalg.norm(p(XY[:,0]) - XY[:,1]))

plt.plot([p.nsplit for p in poly_list], error, '-o')
plt.ylabel('erreur')
plt.xlabel('nombre de divisions')
plt.yscale('log')
plt.grid()

print(error)